In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import datetime
import seaborn as sns
sns.set()
import matplotlib

pd.set_option('max_columns',1000)
pd.set_option('max_rows',1000)


import warnings
warnings.filterwarnings('ignore')

import re
import geocoder
from geopy.distance import great_circle, vincenty
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import os
import gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from time import time
import datetime
from script import RegressionPredictor,LogRegressionPredictor
import japanize_matplotlib
from utils import save_data
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
SEED=1234
n_splits=10

In [17]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
id_max = train.id.max()
df = pd.concat([train, test]).reset_index(drop=True)
print('訓練データ : ', train.shape)
print('テストデータ : ',test.shape)

訓練データ :  (31470, 17)
テストデータ :  (31262, 16)


In [28]:
takuchi_df=pd.read_csv('./team_data/2019_TAKUCHI_k_13_after.csv',encoding='cp932')
station_df=pd.read_csv('./team_data/tokyo_station.csv',index_col=0)
station_df=station_df.rename(columns={'N02_005':'nearest_sta'})
station_df['nearest_sta']=station_df['nearest_sta'].apply(lambda x:x+'駅')


In [59]:
train=train.merge(station_df[['nearest_sta','駅のサイズ(路線数)','乗降人数']],how='left',on='nearest_sta').drop_duplicates(subset='id').reset_index(drop=True)
test=test.merge(station_df[['nearest_sta','駅のサイズ(路線数)','乗降人数']],how='left',on='nearest_sta').drop_duplicates(subset='id').reset_index(drop=True)

In [29]:
train=pd.read_csv('./input/prep_train1105.csv')
test=pd.read_csv('./input/prep_test1105.csv')

In [60]:
takuchi_df

,市区,所在地番,所在地,鑑定評価額,1m2当たりの価格,相続税路線価,利用の現況,地上階数,地下階数,駅名,比準価格,収益価格,積算価格,開発法による価格
0,千代田区,三番町６番２５,東京都千代田区三番町6-25,2.970000e+09,3060000,2380000,共同住宅,10,0,半蔵門駅,3120000,0,0,3020000
1,千代田区,三番町６番２５,東京都千代田区三番町6-25,2.950000e+09,3040000,2380000,共同住宅,10,0,半蔵門駅,3150000,0,0,2930000
2,千代田区,紀尾井町３番２７外,東京都千代田区紀尾井町3-27,1.060000e+09,1990000,1540000,共同住宅,5,0,永田町駅,2120000,1710000,0,0
3,千代田区,紀尾井町３番２７外,東京都千代田区紀尾井町3-27,1.060000e+09,1980000,1540000,共同住宅,5,0,永田町駅,2100000,1700000,0,0
4,千代田区,六番町６番１外,東京都千代田区六番町6-1,8.910000e+09,3930000,2930000,共同住宅,10,1,四ツ谷駅,3970000,0,0,3910000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,江戸川区,中央３丁目１６６８番,東京都江戸川区中央3丁目1668,1.280000e+08,278000,220000,事務所、倉庫兼寮,3,0,新小岩駅,286000,187000,0,0
3184,江戸川区,西一之江４丁目１１９８番２外,東京都江戸川区西一之江4丁目1198-2,8.780000e+07,266000,210000,工場、事務所兼住宅,3,0,一之江駅,273000,183000,0,0
3185,江戸川区,西一之江４丁目１１９８番２外,東京都江戸川区西一之江4丁目1198-2,8.780000e+07,266000,210000,工場、事務所兼住宅,3,0,一之江駅,277000,174000,0,0
3186,江戸川区,臨海町３丁目６番２５,東京都江戸川区臨海町3丁目6-25,7.080000e+08,322000,245000,事務所兼倉庫,3,0,葛西臨海公園駅,335000,230000,0,0
